# Tutorial 4 - A metrological datastream.

In this tutorial we introduce the new metrologically enabled agents. We initialize an
agent, which generates an infinite sine signal. The signal is generated from the
built-in class `MetrologicalSineGenerator` which delivers on each call one timestamp and one value each
with associated uncertainties.
 
The _MetrologicalSineGeneratorAgent_ is based on the new class
_agentMET4FOF.metrological_agents.MetrologicalAgent_. We only adapt the
methods `init_parameters()` and `agent_loop()`. This we need to hand over an instance
of the signal generating class and to generate the actual samples. The rest of the
buffering and plotting logic is encapsulated inside of the new base classes.

In [1]:
# %load tutorial_4_metrological_streams.py
from agentMET4FOF.agents import AgentNetwork
from agentMET4FOF.metrological_agents import MetrologicalAgent, MetrologicalMonitorAgent
from agentMET4FOF.metrological_streams import (
    MetrologicalDataStreamMET4FOF,
    MetrologicalSineGenerator,
)


class MetrologicalSineGeneratorAgent(MetrologicalAgent):
    """An agent streaming a sine signal

    Takes samples from an instance of :py:class:`MetrologicalSineGenerator` pushes
    them sample by sample to connected agents via its output channel.
    """

    # The datatype of the stream will be MetrologicalSineGenerator.
    _stream: MetrologicalDataStreamMET4FOF

    def init_parameters(
        self,
        signal: MetrologicalDataStreamMET4FOF = MetrologicalSineGenerator(),
        **kwargs
    ):
        """Initialize the input data stream

        Parameters
        ----------
        signal : MetrologicalDataStreamMET4FOF
            the underlying signal for the generator
        """
        self._stream = signal
        super().init_parameters()
        self.set_output_data(channel="default", metadata=self._stream.metadata)

    def agent_loop(self):
        """Model the agent's behaviour

        On state *Running* the agent will extract sample by sample the input
        datastream's content and push it via invoking
        :py:method:`AgentMET4FOF.send_output`.
        """
        if self.current_state == "Running":
            self.set_output_data(channel="default", data=[self._stream.next_sample()])
            super().agent_loop()


def demonstrate_metrological_stream():

    # start agent network server
    agent_network = AgentNetwork(dashboard_modules=True)

    # Initialize signal generating class outside of agent framework.
    signal = MetrologicalSineGenerator()

    # Initialize metrologically enabled agent taking name from signal source metadata.
    source_name = signal.metadata.metadata["device_id"]
    source_agent = agent_network.add_agent(
        name=source_name, agentType=MetrologicalSineGeneratorAgent
    )
    source_agent.init_parameters(signal)

    # Initialize metrologically enabled plotting agent.
    monitor_agent = agent_network.add_agent(
        "MonitorAgent", agentType=MetrologicalMonitorAgent
    )

    # agent_network.bind_agents(source_agent, monitor_agent)
    # Bind agents.
    source_agent.bind_output(monitor_agent)

    # Set all agents states to "Running".
    agent_network.set_running_state()

    # Allow for shutting down the network after execution.
    return agent_network


if __name__ == "__main__":
    demonstrate_metrological_stream()

Starting NameServer...
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:3333 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:3333
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "agentMET4FOF.dashboard.Dashboard" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


INFO [2020-12-08 16:35:51.844625] (SineGenerator): INITIALIZED
INFO [2020-12-08 16:35:51.924866] (MonitorAgent): INITIALIZED
[2020-12-08 16:35:51.950078] (SineGenerator): Connected output module: MonitorAgent
SET STATE:   Running
[2020-12-08 16:35:52.851161] (SineGenerator): Pack time: 0.000906
[2020-12-08 16:35:52.856124] (SineGenerator): Sending: [array([[0.        , 0.        , 0.38113488, 0.25      ]]), <time_series_metadata.scheme.MetaData object at 0x7f9c5be41280>]
[2020-12-08 16:35:52.858618] (MonitorAgent): Received: {'from': 'SineGenerator', 'data': array([[0.        , 0.        , 0.38113488, 0.25      ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7f9c5be42430>, 'senderType': 'MetrologicalSineGeneratorAgent', 'channel': 'default'}
[2020-12-08 16:35:52.862388] (MonitorAgent): Buffer: {'SineGenerator': {'data': array([[0.        , 0.        , 0.38113488, 0.25      ]]), 'metadata': [<time_series_metadata.scheme.MetaData object at 0x7f9c5be42430>]}}
[2020-12-0

127.0.0.1 - - [08/Dec/2020 17:35:54] "POST /_dash-update-component HTTP/1.1" 204 -


[2020-12-08 16:35:54.849635] (SineGenerator): Pack time: 0.000496
[2020-12-08 16:35:54.851330] (SineGenerator): Sending: [array([[0.004     , 0.        , 1.08806646, 0.25      ]]), <time_series_metadata.scheme.MetaData object at 0x7f9c5be41280>]
[2020-12-08 16:35:54.852559] (MonitorAgent): Received: {'from': 'SineGenerator', 'data': array([[0.004     , 0.        , 1.08806646, 0.25      ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7f9c4259fa90>, 'senderType': 'MetrologicalSineGeneratorAgent', 'channel': 'default'}
[2020-12-08 16:35:54.856011] (MonitorAgent): Buffer: {'SineGenerator': {'data': array([[0.        , 0.        , 0.38113488, 0.25      ],
       [0.002     , 0.        , 0.38406741, 0.25      ],
       [0.004     , 0.        , 1.08806646, 0.25      ]]), 'metadata': [<time_series_metadata.scheme.MetaData object at 0x7f9c5be42430>, <time_series_metadata.scheme.MetaData object at 0x7f9c5be424f0>, <time_series_metadata.scheme.MetaData object at 0x7f9c4259fa90>

[2020-12-08 16:35:59.856536] (MonitorAgent): Tproc: 0.003695
[2020-12-08 16:36:00.849874] (SineGenerator): Pack time: 0.000584
[2020-12-08 16:36:00.851708] (SineGenerator): Sending: [array([[ 0.016     ,  0.        , -0.32752173,  0.25      ]]), <time_series_metadata.scheme.MetaData object at 0x7f9c5be41280>]
[2020-12-08 16:36:00.852620] (MonitorAgent): Received: {'from': 'SineGenerator', 'data': array([[ 0.016     ,  0.        , -0.32752173,  0.25      ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7f9c5be302e0>, 'senderType': 'MetrologicalSineGeneratorAgent', 'channel': 'default'}
[2020-12-08 16:36:00.857987] (MonitorAgent): Buffer: {'SineGenerator': {'data': array([[ 0.        ,  0.        ,  0.38113488,  0.25      ],
       [ 0.002     ,  0.        ,  0.38406741,  0.25      ],
       [ 0.004     ,  0.        ,  1.08806646,  0.25      ],
       [ 0.006     ,  0.        ,  1.64464102,  0.25      ],
       [ 0.008     ,  0.        ,  0.62548625,  0.25      ],
     

[2020-12-08 16:36:04.852543] (MonitorAgent): Received: {'from': 'SineGenerator', 'data': array([[0.024     , 0.        , 0.92631304, 0.25      ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7f9c5be42490>, 'senderType': 'MetrologicalSineGeneratorAgent', 'channel': 'default'}
[2020-12-08 16:36:04.853209] (SineGenerator): Sending: [array([[0.024     , 0.        , 0.92631304, 0.25      ]]), <time_series_metadata.scheme.MetaData object at 0x7f9c5be41280>]
[2020-12-08 16:36:04.860899] (MonitorAgent): Buffer: {'SineGenerator': {'data': array([[ 0.        ,  0.        ,  0.38113488,  0.25      ],
       [ 0.002     ,  0.        ,  0.38406741,  0.25      ],
       [ 0.004     ,  0.        ,  1.08806646,  0.25      ],
       [ 0.006     ,  0.        ,  1.64464102,  0.25      ],
       [ 0.008     ,  0.        ,  0.62548625,  0.25      ],
       [ 0.01      ,  0.        ,  0.2340912 ,  0.25      ],
       [ 0.012     ,  0.        , -0.75106323,  0.25      ],
       [ 0.014    

[2020-12-08 16:36:07.867344] (MonitorAgent): Tproc: 0.014357
[2020-12-08 16:36:08.849701] (SineGenerator): Pack time: 0.000475
[2020-12-08 16:36:08.851602] (SineGenerator): Sending: [array([[ 0.032     ,  0.        , -0.66261441,  0.25      ]]), <time_series_metadata.scheme.MetaData object at 0x7f9c5be41280>]
[2020-12-08 16:36:08.852274] (MonitorAgent): Received: {'from': 'SineGenerator', 'data': array([[ 0.032     ,  0.        , -0.66261441,  0.25      ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7f9c425a8910>, 'senderType': 'MetrologicalSineGeneratorAgent', 'channel': 'default'}
[2020-12-08 16:36:08.860981] (MonitorAgent): Buffer: {'SineGenerator': {'data': array([[ 0.        ,  0.        ,  0.38113488,  0.25      ],
       [ 0.002     ,  0.        ,  0.38406741,  0.25      ],
       [ 0.004     ,  0.        ,  1.08806646,  0.25      ],
       [ 0.006     ,  0.        ,  1.64464102,  0.25      ],
       [ 0.008     ,  0.        ,  0.62548625,  0.25      ],
     